## Job Aggregator

### Project submission group
- Group member 1
    - Name: Pinkesh Nayak
    - Email: prn33@drexel.edu
- Group member 2
    - Name: Hemachandar Nagarajan
    - Email: hn395@drexel.edu
- Group member 3
    - Name: Sahibzada Aasim Imtiyaz
    - Email: si339@drexel.edu
- Group member 4
    - Name: Pawan Punera
    - Email: pp592@drexel.edu

####  <i> <span style="color:green"> For this project, we are creating a prototype for which we considered only data science related jobs in United States. <br> These both fields, job category and location are customizable. <br> </span> <i>

### Libraries used

In [23]:
import urllib.request
import robotexclusionrulesparser
from bs4 import BeautifulSoup
import requests
import time
import random
import re
from collections import defaultdict
from pprint import pprint
import csv
import dateutil.parser as dateparser
import os
import datetime
import pandas as pd
from dateutil import tz

## Randstad: Web Scraping

#### Checking `robots.txt` file of Staffing firm [Randstad](https://www.randstad.com) to check whether we can scrape the data from its website

In [50]:
url = "https://www.randstad.com/robots.txt"

# Make the request
req = urllib.request.Request(url = url)

# Open the URL
handler = urllib.request.urlopen(req)

# Read/view the data as a string
robots = handler.read().decode('utf-8')
print(robots)

User-agent: *
Sitemap: https://www.randstad.com/sitemap.xml

User-agent: msnbot
Crawl-delay: 5

User-agent: MJ12bot
Crawl-Delay: 5

User-agent: bingbot
Crawl-delay: 5



From the above output Randstad allows scraping of data with crawl delay of 5 seconds.

####  Using `robotexclusionrulesparser.RobotFileParserLookalike()` to confirm that we're allowed to search job page on Randstad.

In [51]:
## spin up the module
rp = robotexclusionrulesparser.RobotFileParserLookalike()

## parse the robots file
print(rp.can_fetch("*", "https://www.randstad.com/jobs/united-states/q-data-science/"))

True


#### Creating directory structures for collection of data from Randstad. 
Storing data in './data/ranstad/randstad.csv'

In [52]:
path = "./data/ranstad/"
randstad_filename = "randstad.csv"
randstand_header = ["company","title","location","link","job_posted","description","category","source"]

if not os.path.exists(path):
    os.makedirs(path)
if not os.path.isfile(path+randstad_filename):
    with open(path+randstad_filename, 'w', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        writer.writerow(randstand_header)

#### Function to fetch required data from a page for a particular job.

In [40]:
def scrapeDataFromPage(page, writer):

    html = requests.get(page).text
    soup2 = BeautifulSoup(html, 'html.parser')

    job_role = soup2.find('h1').find('span').text

    job_details = soup2.find('div', "cf job-company-logo-summary").find('dl')

    dts = {dt.text.strip() : dt.findNext("dd").text.strip() for dt in job_details.find_all('dt')}
    date_posted = dateparser.parse(dts.get('posted'))
    location = dts.get('location')
    job_category = dts.get('job category')
    #job_type = dts.get('job type')
    #working_hours = dts.get('working hours')
    #salary = dts.get('salary')
    source = "Randstad"
    company_name = "RANDSTAD PROFESSIONALS US L P"
    job_description = soup2.find('div', {"id" : "js_description"}).text
    
    row = [company_name,job_role,location,page,date_posted,job_description,job_category,source]
    writer.writerow(row)

#### Function to get link of every jobs present on [search pages](https://www.randstad.com/jobs/united-states/q-data-science/)  and calling above function `scrapeDataFromPage()`.
NOTE: Below fucntion is only called only once to fetch all the data science jobs in US.

In [124]:
def fetchDataFromRandstad():
    country = "united-states"
    job_category = "data-science"

    randstad_url = "https://www.randstad.com/jobs/{}/q-{}/".format(country, job_category)

    #total_jobs = int(soup.find("div", "job-search-header-jobs").find('span').text.split(" jobs")[0])
    crawl_delay = 5
    header = {'User-Agent': 'Mozilla/5.0'}

    next_page = randstad_url

    randstad_part_url = "https://www.randstad.com"
    randstad_filename = "./data/ranstad/randstad.csv"

    with open(randstad_filename, 'a', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')

        while True:
            randstad_html = requests.get(next_page, headers = header).text
            soup = BeautifulSoup(randstad_html, 'html.parser')

            for result_link in soup.find("section", "job-search-results").find_all("header", "cf"):
                scrapeDataFromPage(randstad_part_url + result_link.find("a").get("href"), writer)
                time.sleep(crawl_delay + random.randrange(1, 10))

            is_next = soup.find("a", "results-pager-next-prev icon-arrow-right")

            if is_next:
                next_page = randstad_part_url + soup.find("a", "results-pager-next-prev icon-arrow-right").get("href")
                time.sleep(crawl_delay + random.randrange(1, 10))
            else:
                break

In [125]:
fetchDataFromRandstad()

#### Once we fetch all the data using the above function, we need to update our data.
Ranstad provides [RSS feeds](https://www.randstad.com/jobs/rss/united-states/q-data-science/) of the recent updated data. 
We can create python script file with below code and set a cron utility<br> `@hourly python3 fetchDataFromRandstadRSS.py` to perform our scraping once hourly from the RSS feeds. <br> We can compare `pubDate` present in feeds with the time the cron utility last ran and only fetch the latest records link and call function `scrapeDataFromPage()`.

In [127]:
def fetchDataFromRandstadRSS():
    country = "united-states"
    job_category = "data-science"
    crawl_delay = 5

    randstad_rss_url = "https://www.randstad.com/jobs/rss/{}/q-{}/".format(country, job_category)
    current_time = datetime.datetime.now()
    previous_hour = current_time - datetime.timedelta(hours=1)

    previous_hour_utc = previous_hour.astimezone(datetime.timezone.utc)

    randstad_rss_html = requests.get(randstad_rss_url).text
    rss_soup = BeautifulSoup(randstad_rss_html, 'html.parser')
    randstad_filename = "./data/ranstad/randstad.csv"

    with open(randstad_filename, 'a', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        for item in rss_soup.find_all("item"):
            pubdate = item.find("pubdate").text
            pubdate = dateparser.parse(pubdate)
            if pubdate >= previous_hour_utc:
                rss_link = item.find("link").next_sibling.string.strip()
                scrapeDataFromPage(rss_link, writer)
                time.sleep(crawl_delay + random.randrange(1, 5))

In [128]:
fetchDataFromRandstadRSS()

## Adzuna API

#### After checking [Terms of use](https://developer.adzuna.com/docs/terms_of_service) and [Adzuna API page](https://developer.adzuna.com/overview), we found that there are no rate limits for using its API. Also, it states that it may be used for personal or academic research. We have created developer account and recieved an `app_key` and `aap_id` to access the API.

#### Creating directory structures for collection of data from Adzuna. 
Storing data in './data/adzuna/adzuna.csv'

In [129]:
path = "./data/adzuna/"
adzuna_filename = "adzuna.csv"
adzuna_header = ["company","title","location","link","job_posted","description","category","source"]
 
if not os.path.exists(path):
    os.makedirs(path)
if not os.path.isfile(path+adzuna_filename):
    with open(path+adzuna_filename, 'w', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        writer.writerow(adzuna_header)

#### Defining fucntion `fetchDataFromAdzuna()` to get job listings from its [API](https://api.adzuna.com/v1/api/jobs/us/search/1?app_id=a4f5ad31&app_key=34579444c00900f8fa58b004dc308aff&what=data%20science&where=us).
Checking for all the number of pages present and fetching all job listings from each page.

In [130]:
def fetchDataFromAdzuna(max_days_old = 0):
    
    country = "us"
    APP_id = 'a4f5ad31'
    APP_key = '34579444c00900f8fa58b004dc308aff'
    what = "data science"
    where = "us"
    adzuna_url = "https://api.adzuna.com/v1/api/jobs/{}/search/{}?app_id={}&app_key={}&what={}&where={}&max_days_old={}"
    response = requests.get(adzuna_url.format(country,1,APP_id, APP_key, what, where,max_days_old))
    results= response.json()
    total_jobs = results.get('count')
    time.sleep(5)
    
    with open("./data/adzuna/adzuna.csv", 'a', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
    
        for page in range(1,total_jobs+1):
            response = requests.get(adzuna_url.format(country,page,APP_id, APP_key, what, where,max_days_old))
            results= response.json()
            for result in results['results']:
                if result.get('title'):
                    title = BeautifulSoup(result['title']).get_text()
                if result.get('description'):
                    description = BeautifulSoup(result['description']).get_text()
                row_values = [result['company'].get('display_name'),
                                      title,
                                      result['location'].get('display_name'),
                                      result['redirect_url'],
                                      dateparser.parse(result["created"]),
                                      description,
                                      result['category']['label'],
                                      'Adzuna']

                try:
                    writer.writerow(row_values)
                except UnicodeEncodeError:
                    row = [item.encode('utf-8') if item and not isinstance(item, datetime.datetime) else item for item in row_values]
                    writer.writerow(row)

            time.sleep(5)

In [131]:
fetchDataFromAdzuna()

#### Once we fetch all the data using the above function, we need to update our data.
Adzuna provides parameter `max_days_old` in its API which can be used to get latest data by providing the number of days, so if we give value as 1, it will fetch all data from previous day till today.
We can create python script file which includes the above function `fetchDataFromAdzuna()`  and set a cron utility<br> `@daily python3 fetchDataFromAdzuna.py` to fetch updated data daily from the API.

In [132]:
fetchDataFromAdzuna(1)

## GitHub API

#### We checked [Terms of use](https://help.github.com/en/github/site-policy/github-terms-of-service#h-api-terms). GitHub's API  doesn't require an "access token". 

#### Creating directory structures for collection of data from GitHub. 
Storing data in './data/github/github.csv'

In [133]:
path = "./data/github/"
github_filename = "github.csv"
github_header = ["company","title","location","link","job_posted","description","category","source"]

if not os.path.exists(path):
    os.makedirs(path)
if not os.path.isfile(path+github_filename):
    with open(path+github_filename, 'w', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        writer.writerow(github_header)

#### Defining fucntion `fetchDataFromGithub()` to get job listings from its [API](https://jobs.github.com/positions.json?description=data+science&location=US).
Checking for all the number of pages present and fetching all job listings from each page.

In [134]:
def fetchDataFromGithub():
    
    location = "us"
    job = "data science"
    github_url = "https://jobs.github.com/positions.json?{}&location={}&page={}"
    
    with open("./data/github/github.csv", 'a', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
    
        page_count = 1
    
        while True:
            response = requests.get(github_url.format(job,location,page_count))
            results = response.json()
            
            if len(results) != 0:
                page_count += 1
                for result in results:
                    if result.get('title'):
                        title = BeautifulSoup(result['title']).get_text()
                    if result.get('description'):
                        description = BeautifulSoup(result['description']).get_text()
                    row_values = [result['company'],
                                          title,
                                          result['location'],
                                          result['url'],
                                          dateparser.parse(result["created_at"]),
                                          description,
                                          "",
                                          'GitHub']

                    try:
                        writer.writerow(row_values)
                    except UnicodeEncodeError:
                        row = [item.encode('utf-8') if item and not isinstance(item, datetime.datetime) else item for item in row_values]
                        writer.writerow(row)
                time.sleep(5)
            else:
                break

In [135]:
fetchDataFromGithub()

## The Muse API

#### After checking [Terms of use](https://www.themuse.com/developers/api/v2/termse) and [The Muse API page](https://www.themuse.com/developers/api/v2), If we don't register our app, we are limited to 500 requests per hour. We have created developer account and recieved an `app_key` which will allow us to make up to 3600 requests per hour.

#### Creating directory structures for collection of data from 'The Muse'. 
Storing data in './data/muse/muse.csv'

In [136]:
path = "./data/muse/"
muse_filename = "muse.csv"
muse_header = ["company","title","location","link","job_posted","description","category","source"]

if not os.path.exists(path):
    os.makedirs(path)
if not os.path.isfile(path+muse_filename):
    with open(path+muse_filename, 'w', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        writer.writerow(muse_header)

#### Defining fucntion `fetchDataFromMuse()` to get job listings from its [API](https://www.themuse.com/api/public/jobs?category=Data%20Science&location=United%20States&page=1).
Checking for all the number of pages present and fetching all job listings from each page.

In [137]:
def fetchDataFromMuse():
    
    what = "Data Science"
    location = "United States"
    APP_key = 'aee960065c38dee83e8a6f9af40f86c7bb89843d3faf60230066b36418cf6257'
    muse_url = "https://www.themuse.com/api/public/jobs?category={}&location={}&page={}&app_key={}"
    response = requests.get(muse_url.format(what,location,1,APP_key))
    request_per_hour = int(response.headers['X-RateLimit-Remaining'])
    results= response.json()
    page_count = results.get('page_count')
    time.sleep(5)
    
    with open("./data/muse/muse.csv", 'a', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
    
        for page in range(1,page_count+1):
            if page < request_per_hour:
                response = requests.get(muse_url.format(what,location,page,APP_key))
                results= response.json()
                for result in results['results']:
                    if result.get('contents'):
                        description = BeautifulSoup(result['contents']).get_text()
                    else:
                        description = ""
                    row_values = [result['company'].get('name'),
                                          result.get('name'),
                                          result['locations'][0].get('name'),
                                          result['refs'].get('landing_page'),
                                          dateparser.parse(result["publication_date"]),
                                          description,
                                          result['categories'][0].get('name'),
                                          'The Muse']

                    try:
                        writer.writerow(row_values)
                    except UnicodeEncodeError:
                        row = [item.encode('utf-8') if item and not isinstance(item, datetime.datetime) else item for item in row_values]
                        writer.writerow(row)
            else:
                break

            time.sleep(5)

In [138]:
fetchDataFromMuse()

## United States Citizenship and Immigration Services (USCIS) H1B data

We are fetching 2019 H-1B Employer Data which is openly available on [USCIS Website](https://www.uscis.gov/tools/reports-studies/h-1b-employer-data-hub-files)

In [139]:
def fetchDataFromUSCIS():
    path = "./data/uscis/"
    uscis_filename = "uscis.csv"
    uscis_url = "https://www.uscis.gov/sites/default/files/USCIS/Data/Employment-based/H-1B/h1b_datahubexport-2019.csv"

    if not os.path.exists(path):
        os.makedirs(path)

    uscis_response = requests.get(uscis_url)
    uscis_text = uscis_response.text.strip().split("\n")
    uscis_reader = csv.reader(uscis_text)
    uscis_data = list(uscis_reader)
    with open(path+uscis_filename, 'w', newline='') as file_handle:
        writer = csv.writer(file_handle, delimiter=',')
        writer.writerows(uscis_data)

In [140]:
fetchDataFromUSCIS()

#### Displaying Randstad jobs data which we stored in the directory.

In [2]:
randstad = pd.read_csv("./data/ranstad/randstad.csv", sep = ",", header = 0, parse_dates = [4], encoding = "ISO-8859-1")
randstad.head()

,company,title,location,link,job_posted,description,category,source
0,RANDSTAD PROFESSIONALS US L P,sr. data scientist,new york,https://www.randstad.com/jobs/united-states/sr...,2020-06-04,"job summary:Our client is looking for dynamic,...",Information Technology,Randstad
1,RANDSTAD PROFESSIONALS US L P,business intelligence analyst,"kansas city, missouri",https://www.randstad.com/jobs/united-states/bu...,2020-06-05,job summary:Randstad Technologies is looking f...,Information Technology,Randstad
2,RANDSTAD PROFESSIONALS US L P,people research scientist,"menlo park, california",https://www.randstad.com/jobs/united-states/pe...,2020-05-26,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad
3,RANDSTAD PROFESSIONALS US L P,people research scientist,new york,https://www.randstad.com/jobs/united-states/pe...,2020-05-26,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad
4,RANDSTAD PROFESSIONALS US L P,people research scientist,"seattle, washington",https://www.randstad.com/jobs/united-states/pe...,2020-05-26,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad


#### Displaying Adzuna jobs data which we stored in the directory.

In [3]:
adzuna = pd.read_csv("./data/adzuna/adzuna.csv", sep = ",", header = 0, parse_dates = [4], encoding = "ISO-8859-1")
adzuna.head()

,company,title,location,link,job_posted,description,category,source
0,Home Depot,Data Scientist,"Houston, Texas County",https://www.adzuna.com/land/ad/1373178326?se=Y...,2019-12-13 13:57:49+00:00,POSITION PURPOSE A Data Scientist leverages th...,IT Jobs,Adzuna
1,Home Depot,"Data Scientist, Pricing","Atlanta, Fulton County",https://www.adzuna.com/land/ad/1494906867?se=Y...,2020-03-18 15:45:03+00:00,Position Purpose A Data Scientist leverages th...,IT Jobs,Adzuna
2,Home Depot,Associate Data Scientist,"Atlanta, Fulton County",https://www.adzuna.com/land/ad/1571930776?se=Y...,2020-06-12 19:38:04+00:00,POSITION PURPOSE The Associate Data Scientist ...,IT Jobs,Adzuna
3,Home Depot,Data Scientist - Supply Chain,"Atlanta, Fulton County",https://www.adzuna.com/land/ad/1534472383?se=Y...,2020-05-01 19:07:09+00:00,POSITION PURPOSE: A Data Scientist leverages t...,IT Jobs,Adzuna
4,Home Depot,Principal Engineer - Data Science,"Atlanta, Fulton County",https://www.adzuna.com/land/ad/1430632954?se=Y...,2020-01-31 14:43:08+00:00,Position Purpose : A Principal Data Scientist ...,IT Jobs,Adzuna


#### Displaying GitHub jobs data which we stored in the directory.

In [4]:
github = pd.read_csv("./data/github/github.csv", sep = ",", header = 0, parse_dates = [4], encoding = "ISO-8859-1")
github.head()

,company,title,location,link,job_posted,description,category,source
0,BlueVolt,Senior Software Engineer (.NET),"Portland, OR, US",https://jobs.github.com/positions/8db5942d-34b...,2020-06-12 21:52:25+00:00,Position Summary\nBlueVolt is growing. We have...,NaN,GitHub
1,Lawrence Berkeley National Laboratory,Web Developer (The Materials Project),Berkeley,https://jobs.github.com/positions/f1759f8d-107...,2020-06-12 21:15:38+00:00,Web Developer (The Materials Project) - 90287\...,NaN,GitHub
2,Game Closure,Senior Game Engineer (REMOTE),"San Francisco, Mountain View, Tokyo, Remote",https://jobs.github.com/positions/e9e632a7-c75...,2019-08-03 00:50:23+00:00,"GAME CLOSURE\nSan Francisco, Mountain View, To...",NaN,GitHub
3,Defendify,Full Stack Engineer,Maine,https://jobs.github.com/positions/7e5a0fb2-778...,2020-06-05 18:53:43+00:00,Thanks for your interest in working with us! D...,NaN,GitHub
4,Kasisto,Front-End Software Engineer,New York,https://jobs.github.com/positions/90b7a228-a21...,2020-06-03 19:36:58+00:00,Humanizing Digital Experiences®\nKasistos Dig...,NaN,GitHub


#### Displaying 'The Muse' jobs data which we stored in the directory.

In [5]:
muse = pd.read_csv("./data/muse/muse.csv", sep = ",", header = 0, parse_dates = [4], encoding = "ISO-8859-1")
muse.head()

,company,title,location,link,job_posted,description,category,source
0,Advanced Group,SAS Programmer,Flexible / Remote,https://www.themuse.com/jobs/advancedgroup/sas...,2020-05-15 11:13:13.282083+00:00,Advanced Clinical is a global clinical develop...,Data Science,The Muse
1,The Knot Worldwide,Head of Data Engineering,United States,https://www.themuse.com/jobs/theknotworldwide/...,2020-05-04 23:31:13.547738+00:00,WHAT WE DO MATTERS:\nHere at The Knot Worldwid...,Data Science,The Muse
2,b'Collibra',b'Salesforce Developer',"b'Atlanta, GA'",b'https://www.themuse.com/jobs/collibra/salesf...,2020-06-09 23:02:03.711114+00:00,"b""We're Ushering a New Era of Data Participati...",b'Data Science',b'The Muse'
3,Invitae,Biopharma Partnerships Program Manager,Flexible / Remote,https://www.themuse.com/jobs/invitae/biopharma...,2020-06-05 23:02:33.443646+00:00,Invitae is a rapidly growing genetic informati...,Data Science,The Muse
4,Medium,"Head of Data Science, Insights & Analytics",Flexible / Remote,https://www.themuse.com/jobs/medium/head-of-da...,2020-06-03 13:57:29.821055+00:00,Mediums mission is to help people deepen thei...,Data Science,The Muse


#### Displaying USCIS 2019 H-1B employer data which we stored in the directory.

In [6]:
uscis = pd.read_csv("./data/uscis/uscis.csv", sep = ",", header = 0, encoding = "ISO-8859-1")
uscis.head()

,Fiscal Year,Employer,Initial Approvals,Initial Denials,Continuing Approvals,Continuing Denials,NAICS,Tax ID,State,City,ZIP
0,2019,SOUTHERN CARPET HARDWOOD & TILE IN,1,0,0,0,23,NaN,AL,BIRMINGHAM,35209.0
1,2019,UAB HEALTH SYSTEM,0,0,0,1,56,NaN,AL,BIRMINGHAM,35233.0
2,2019,BIRMINGHAM VA MEDICAL CENTER,0,0,1,0,62,NaN,AL,BIRMINGHAM,35233.0
3,2019,GESTAMP ALABAMA LLC,1,0,0,0,33,NaN,AL,MC CALLA,35111.0
4,2019,ARKANSAS HEALTH GROUP,0,0,1,0,62,NaN,AR,LITTLE ROCK,72211.0


#### Concatenating all the jobs data collected from `Randstad`, `Adzuna`, `GitHub`, and `The Muse`.

In [7]:
job_data = pd.concat([randstad, adzuna, github, muse])

#### Dropping duplicate job records.

In [8]:
job_data = job_data.drop_duplicates(subset=job_data.columns.difference(['source']), keep = "first")
job_data = job_data.reset_index()
job_data.drop('index', axis=1, inplace=True)
job_data.tail()

,company,title,location,link,job_posted,description,category,source
22189,Socure,Senior Data Scientist,Flexible / Remote,https://www.themuse.com/jobs/socure/senior-dat...,2020-06-11 19:47:18.922553+00:00,"\n\n\n\nFounded in 2012, Socure is the leader ...",Data Science,The Muse
22190,Advanced Group,Senior Clinical Data Coordinator,Flexible / Remote,https://www.themuse.com/jobs/advancedgroup/sen...,2020-05-23 11:09:10.789013+00:00,Advanced Clinical is a global clinical develop...,Data Science,The Muse
22191,PepsiCo,eCommerce Business Intelligence Intern,United States,https://www.themuse.com/jobs/pepsico/ecommerce...,2020-05-12 21:13:42.378591+00:00,Auto req ID: 191272BR Job Description We are ...,Data Science,The Muse
22192,Guidewire,"Actuarial Consultant, Remote USA",Flexible / Remote,https://www.themuse.com/jobs/guidewire/actuari...,2020-05-29 18:44:57.974847+00:00,\n\nResponsibilities:This position requires ad...,Data Science,The Muse
22193,Invitae,"Bioinformatics Engineer, Algorithm",Flexible / Remote,https://www.themuse.com/jobs/invitae/bioinform...,2020-04-30 23:02:35.384368+00:00,\nInvitae is a healthcare technology company ...,Data Science,The Muse


#### Creating below function `mergeWithUSCISData()` which merges jobs data with USCIS data to get the H1B statistics for each company in jobs data.
Since company name present in jobs data does not match with USCIS Employer name (Registered Name). We are using 'contains()' function.

In [11]:
def mergeWithUSCISData(company):
    if uscis['Employer'].str.contains(company, na=False, regex=False).any():
        row = uscis[uscis['Employer'].str.contains(company, na=False, regex=False, case = False)].iloc[0]

        return pd.Series({
            'initial_approvals_2019': row['Initial Approvals'],
            'initial_denials_2019': row['Initial Denials'],
            'continuing_approvals_2019': row['Continuing Approvals'],
            'continuing_denials_2019': row['Continuing Denials']
        })
    else:
        return pd.Series({
            'initial_approvals_2019': 0,
            'initial_denials_2019': 0,
            'continuing_approvals_2019': 0,
            'continuing_denials_2019': 0
        })

In [12]:
uscis['Employer'] = uscis['Employer'].str.lower()
company_details = job_data['company'].str.lower().apply(mergeWithUSCISData)
job_uscis = pd.concat([job_data, company_details],axis=1)

#### Checking for NULL values.

In [15]:
job_uscis.isnull().any()

company                       True
title                        False
location                     False
link                         False
job_posted                   False
description                   True
category                      True
source                       False
initial_approvals_2019       False
initial_denials_2019         False
continuing_approvals_2019    False
continuing_denials_2019      False
dtype: bool

#### Dropping the rows where company name is NULL. We are not dropping NAs for  'description' and 'category' variables as since it not that important, also we have the link where user can see that particular job posting.

In [56]:
job_uscis = job_uscis.dropna(subset=['company'])

In [57]:
job_uscis

,company,title,location,link,job_posted,description,category,source,initial_approvals_2019,initial_denials_2019,continuing_approvals_2019,continuing_denials_2019
0,RANDSTAD PROFESSIONALS US L P,sr. data scientist,new york,https://www.randstad.com/jobs/united-states/sr...,2020-06-04 00:00:00,"job summary:Our client is looking for dynamic,...",Information Technology,Randstad,0,0,1,0
1,RANDSTAD PROFESSIONALS US L P,business intelligence analyst,"kansas city, missouri",https://www.randstad.com/jobs/united-states/bu...,2020-06-05 00:00:00,job summary:Randstad Technologies is looking f...,Information Technology,Randstad,0,0,1,0
2,RANDSTAD PROFESSIONALS US L P,people research scientist,"menlo park, california",https://www.randstad.com/jobs/united-states/pe...,2020-05-26 00:00:00,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad,0,0,1,0
3,RANDSTAD PROFESSIONALS US L P,people research scientist,new york,https://www.randstad.com/jobs/united-states/pe...,2020-05-26 00:00:00,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad,0,0,1,0
4,RANDSTAD PROFESSIONALS US L P,people research scientist,"seattle, washington",https://www.randstad.com/jobs/united-states/pe...,2020-05-26 00:00:00,job summary:Are you a PhD or Master with exper...,Human Resources,Randstad,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
22189,Socure,Senior Data Scientist,Flexible / Remote,https://www.themuse.com/jobs/socure/senior-dat...,2020-06-11 19:47:18.922553+00:00,"\n\n\n\nFounded in 2012, Socure is the leader ...",Data Science,The Muse,0,0,5,0
22190,Advanced Group,Senior Clinical Data Coordinator,Flexible / Remote,https://www.themuse.com/jobs/advancedgroup/sen...,2020-05-23 11:09:10.789013+00:00,Advanced Clinical is a global clinical develop...,Data Science,The Muse,0,0,0,0
22191,PepsiCo,eCommerce Business Intelligence Intern,United States,https://www.themuse.com/jobs/pepsico/ecommerce...,2020-05-12 21:13:42.378591+00:00,Auto req ID: 191272BR Job Description We are ...,Data Science,The Muse,6,0,28,3
22192,Guidewire,"Actuarial Consultant, Remote USA",Flexible / Remote,https://www.themuse.com/jobs/guidewire/actuari...,2020-05-29 18:44:57.974847+00:00,\n\nResponsibilities:This position requires ad...,Data Science,The Muse,13,6,33,3


## Final Job Data

In [58]:
print(job_uscis.info())
job_uscis.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22191 entries, 0 to 22193
Data columns (total 12 columns):
company                      22191 non-null object
title                        22191 non-null object
location                     22191 non-null object
link                         22191 non-null object
job_posted                   22191 non-null object
description                  22190 non-null object
category                     22105 non-null object
source                       22191 non-null object
initial_approvals_2019       22191 non-null object
initial_denials_2019         22191 non-null object
continuing_approvals_2019    22191 non-null object
continuing_denials_2019      22191 non-null object
dtypes: object(12)
memory usage: 2.2+ MB
None


,company,title,location,link,job_posted,description,category,source,initial_approvals_2019,initial_denials_2019,continuing_approvals_2019,continuing_denials_2019
22189,Socure,Senior Data Scientist,Flexible / Remote,https://www.themuse.com/jobs/socure/senior-dat...,2020-06-11 19:47:18.922553+00:00,"\n\n\n\nFounded in 2012, Socure is the leader ...",Data Science,The Muse,0,0,5,0
22190,Advanced Group,Senior Clinical Data Coordinator,Flexible / Remote,https://www.themuse.com/jobs/advancedgroup/sen...,2020-05-23 11:09:10.789013+00:00,Advanced Clinical is a global clinical develop...,Data Science,The Muse,0,0,0,0
22191,PepsiCo,eCommerce Business Intelligence Intern,United States,https://www.themuse.com/jobs/pepsico/ecommerce...,2020-05-12 21:13:42.378591+00:00,Auto req ID: 191272BR Job Description We are ...,Data Science,The Muse,6,0,28,3
22192,Guidewire,"Actuarial Consultant, Remote USA",Flexible / Remote,https://www.themuse.com/jobs/guidewire/actuari...,2020-05-29 18:44:57.974847+00:00,\n\nResponsibilities:This position requires ad...,Data Science,The Muse,13,6,33,3
22193,Invitae,"Bioinformatics Engineer, Algorithm",Flexible / Remote,https://www.themuse.com/jobs/invitae/bioinform...,2020-04-30 23:02:35.384368+00:00,\nInvitae is a healthcare technology company ...,Data Science,The Muse,1,0,8,0


#### Creating directory structures for storing final job data. 
Storing data in './data/job_database/job_database.csv'

In [59]:
path = "./data/job_database/"
job_data_filename = 'job_database.csv'

if not os.path.exists(path):
    os.makedirs(path)
job_uscis.to_csv(path+job_data_filename, index=False)

###  Function for the User to query for jobs.
Filters availabe:<br>
`title`: Job title.<br>
`location`: Job location.<br>
`job_posted`: Date of job posting, will return all the records from that date to current date.<br>

In [60]:
def searchForJobs(title = '', location = '', job_posted=''):
    jobs = pd.read_csv("./data/job_database/job_database.csv", sep = ",", header = 0, parse_dates = [4])
    if title:
        jobs = jobs[jobs['title'].str.contains('|'.join(title.split()), case = False, na=False)]
    if location:
        jobs = jobs[jobs['location'].str.contains('|'.join(location.split()), case = False, na=False)]
    if job_posted:
        #jobs['job_posted'] = jobs['job_posted'].apply(lambda x:dateparser.parse(x).replace(tzinfo=None))
        jobs['job_posted'] = jobs['job_posted'].apply(lambda x:x.replace(tzinfo=None))
        jobs = jobs[jobs['job_posted'] >= pd.to_datetime(job_posted)]
        jobs = jobs.sort_values(by=['initial_approvals_2019', 'continuing_approvals_2019'], ascending=False)
    return jobs

### Searching for data scientist jobs in philadelphia which were posted from June 1, 2020 to current date.

In [61]:
searchForJobs('Data Scientist', 'Philadelphia', '2020-06-01')

,company,title,location,link,job_posted,description,category,source,initial_approvals_2019,initial_denials_2019,continuing_approvals_2019,continuing_denials_2019
4993,iSolvers Inc,Data Modeler,"Philadelphia County, Pennsylvania",https://www.adzuna.com/land/ad/1568369498?se=y...,2020-06-09 10:22:07,-models for analytics pipelines andor data-sci...,IT Jobs,Adzuna,11,0,9,2
9346,iSolvers Inc,Data Modeler,"Philadelphia County, Pennsylvania",https://www.adzuna.com/land/ad/1572433263?se=q...,2020-06-13 09:03:52,-models for analytics pipelines andor data-sci...,IT Jobs,Adzuna,11,0,9,2
4032,GlaxoSmithKline,Director Supply Chain Data Science & Network O...,"Philadelphia County, Pennsylvania",https://www.adzuna.com/land/ad/1569394434?se=D...,2020-06-10 06:25:49,Job description Site Name: USA - Pennsylvania ...,"Energy, Oil & Gas Jobs",Adzuna,10,1,32,3
5231,Resource Informatics Group,BigData Architect,"Philadelphia County, Pennsylvania",https://www.adzuna.com/land/ad/1560172723?se=4...,2020-06-01 03:46:32,... quality efforts. bull Work closely with t...,IT Jobs,Adzuna,8,22,9,11
1961,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560834256?se=v...,2020-06-01 17:35:23,Delaware County Data Science Tutor Jobs Varsit...,Teaching Jobs,Adzuna,3,0,0,0
1962,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560834967?se=v...,2020-06-01 17:35:30,Chester County Data Science Tutor Jobs Varsity...,Teaching Jobs,Adzuna,3,0,0,0
1963,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560836520?se=v...,2020-06-01 17:35:45,Montgomery County Data Science Tutor Jobs Vars...,Teaching Jobs,Adzuna,3,0,0,0
1964,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560837132?se=v...,2020-06-01 17:35:52,Radnor Data Science Tutor Jobs Varsity Tutors ...,Teaching Jobs,Adzuna,3,0,0,0
1965,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560837654?se=v...,2020-06-01 17:36:01,West Chester Data Science Tutor Jobs Varsity T...,Teaching Jobs,Adzuna,3,0,0,0
1966,Varsity Tutors,Data Science Tutor Jobs,"Middle West, Philadelphia County",https://www.adzuna.com/land/ad/1560837748?se=v...,2020-06-01 17:36:03,Springfield Data Science Tutor Jobs Varsity Tu...,Teaching Jobs,Adzuna,3,0,0,0
